Grupo 21: Ignacio de la Cruz Crespo y Sergio José Gómez Cortés

## Apartado A

*Dividimos el conjunto de datos en 60% y 40% para train y test, respectivamente*
Creamos una bolsa de palabras con:
- binary = False
- diccionario = words.txt
- stop_words = 'english'
- n_gramas = (1,1)
---

In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics.pairwise import cosine_similarity
train_data = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
test_data = fetch_20newsgroups(subset='test')

print("Training texts:", len(train_data.data))
print("Test texts:", len(test_data.data))

Training texts: 11314
Test texts: 7532


In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

with open('words.txt') as f:
    dictionary = f.read().splitlines()

vectorizer = CountVectorizer(vocabulary=dictionary, stop_words='english',ngram_range=(1,1), binary=False)  

In [3]:
#Cojemos 3 mensajes de cada grupo y los metemos en un dataFrame

import numpy as np
import pandas as pd
cat = fetch_20newsgroups().target_names
tipo = np.array("")
text = np.array("")
for i in cat:
    for j in range(4,7):
        tipo = np.append(tipo,i)
        text = np.append(text,fetch_20newsgroups(categories=[i]).data[j])
threemessage = {'Text': text,
        'Type': tipo
        }
df = pd.DataFrame(threemessage, columns = ['Text', 'Type'])
print(df)

                                                 Text                Type
0                                                                        
1   From: I3150101@dbstu1.rz.tu-bs.de (Benedikt Ro...         alt.atheism
2   From: bil@okcforum.osrhe.edu (Bill Conner)\nSu...         alt.atheism
3   From: keith@cco.caltech.edu (Keith Allan Schne...         alt.atheism
4   From: clldomps@cs.ruu.nl (Louis van Dompselaar...       comp.graphics
..                                                ...                 ...
56  From: dreitman@oregon.uoregon.edu (Daniel R. R...  talk.politics.misc
57  From: wdstarr@athena.mit.edu (William December...  talk.politics.misc
58  From: agr00@ccc.amdahl.com (Anthony G Rose)\nS...  talk.religion.misc
59  From: dotsonm@dmapub.dma.org (Mark Dotson)\nSu...  talk.religion.misc
60  From: sandvik@newton.apple.com (Kent Sandvik)\...  talk.religion.misc

[61 rows x 2 columns]


In [4]:
#Ésta funcion devuelve los 3 mensajes del tema pasado por parametro
def get3message(tipo):
    select_col = df.loc[df['Type'] == tipo]
    return select_col['Text']

In [5]:
#Devuelve la precision media para la clase pasada en category
def calculoPrecision(nivelExhaustividad,category, dataCosine):
    m = np.array([])
    for i in range(len(dataCosine)):
        m = np.append(m,[np.mean(dataCosine[i])])  #Hago la media del resultado de cosine similarity para los 3 mensajes               
    m = np.flip(np.argsort(m)) #'m' contiene las posiciones de train_data de menor a mayor distancia del coseno, asi que le damos la vuelta
    #Este bucle mira las n = nivelExhaustividad primeras posiciones de m y las compara con la categoria que debe ser,
    # para calcular el porcentaje de acierto
    precision = 0
    for j in range(nivelExhaustividad):
        if (train_data.target_names[train_data.target[m[j]]] == category):
            precision = precision + 1
    return precision/nivelExhaustividad*100

In [15]:
train_vector_data = vectorizer.transform(train_data.data) #Vectorizamos train data
for i in range (len(fetch_20newsgroups().target_names)):
    #Calculamos la distancia del coseno entre train_vector_data y get3message(target_names), es decir, la distancia
    #del coseno entre los mensajes de entrenamiento y los de consulta
    test = cosine_similarity(train_vector_data, vectorizer.transform(get3message(fetch_20newsgroups().target_names[i])))
    print("Categoria: " + fetch_20newsgroups().target_names[i])
    print("Precision con nivel exhaustividad 10: "+str(calculoPrecision(10,fetch_20newsgroups().target_names[i],test)))
    print("Precision con nivel exhaustividad 3: "+str(calculoPrecision(3,fetch_20newsgroups().target_names[i],test)))
    print("---------------------------")

Categoria: alt.atheism
Precision con nivel exhaustividad 10: 80.0
Precision con nivel exhaustividad 3: 100.0
---------------------------
Categoria: comp.graphics
Precision con nivel exhaustividad 10: 70.0
Precision con nivel exhaustividad 3: 100.0
---------------------------
Categoria: comp.os.ms-windows.misc
Precision con nivel exhaustividad 10: 90.0
Precision con nivel exhaustividad 3: 100.0
---------------------------
Categoria: comp.sys.ibm.pc.hardware
Precision con nivel exhaustividad 10: 40.0
Precision con nivel exhaustividad 3: 100.0
---------------------------
Categoria: comp.sys.mac.hardware
Precision con nivel exhaustividad 10: 80.0
Precision con nivel exhaustividad 3: 100.0
---------------------------
Categoria: comp.windows.x
Precision con nivel exhaustividad 10: 40.0
Precision con nivel exhaustividad 3: 100.0
---------------------------
Categoria: misc.forsale
Precision con nivel exhaustividad 10: 50.0
Precision con nivel exhaustividad 3: 100.0
---------------------------


**¿Hay muchas diferencias entre los valores de precisión medios para las distintas clases del conjunto de datos? ¿A qué crees que se deben?**

Podemos encontrar una gran diferencia entre algunos topicos, notablemente entre motorcycles y politics.guns, variando de un 100% a un 30% respectivamente. Esto puede ser debido a la existencia de palabras clave en el mundo de las motocicletas que no se comparten fuera de ese ambito. Del mismo modo encontramos una notable igualdad entre todas los calculos con exhaustividad 3, que se deben a la poca piscina de muestras con las que se han realizado, y por lo tanto no se pueden considerar representativas.

---
*A continuación identificamos la clase que haya tenido peores resultados de precisión y mostramos algun mensaje que recuperó erróneamente en las primeras X posiciones.*



In [13]:
#Similar a calculo precision, hacemos un print de los mensajes que han sido erroneos dentro de las nivelExhaustividad primeras
#posiciones
def mensajesErroneos(nivelExhaustividad,category, dataCosine):
    m = np.array([])
    for i in range(len(dataCosine)):
        m = np.append(m,[np.mean(dataCosine[i])])                 
        #print(m)
    m = np.flip(np.argsort(m))
    for j in range(nivelExhaustividad):
        if (train_data.target_names[train_data.target[m[j]]] != category):
            print(train_data.data[m[j]])
            print(train_data.target_names[train_data.target[m[j]]])
    return 0

In [14]:
    #talk.politics.guns es la clase con peor resultado (30%) como se ve previamente
    test2 = cosine_similarity(train_vector_data, vectorizer.fit_transform(get3message("talk.politics.guns")))
    print("Categoria: " + "talk.politics.guns")
    mensajesErroneos(10,"talk.politics.guns",test2)

Categoria: talk.politics.guns
From: matmcinn@nuscc.nus.sg (Matthew MacIntyre at the National University of Senegal)
Subject: Re: Opel owners?
Organization: National University of Singapore
Lines: 1
X-Newsreader: Tin 1.1 PL4

By the way, what do people think about the Opel CAlibra?

rec.autos
From: ee152fcs@sdcc15.ucsd.edu (Bjorn Karlsson)
Subject: Re: WC 93: Results, April 20
Organization: University of California, San Diego
Lines: 12
Nntp-Posting-Host: sdcc15.ucsd.edu

In article <1993Apr21.073134.5117@ericsson.se> etxonss@ufsa.ericsson.se (Staffan Axelsson) writes:
>
> 1993 World Championships in Germany:
> ====================================

Is there any games being shown here in the US from the WC???

Thanks

mc



rec.sport.hockey
From: hungjenc@usc.edu (Hung-Jen Chen)
Subject: test
Article-I.D.: phakt.1pqgltINN9dg
Distribution: na
Organization: University of Southern California, Los Angeles, CA
                test
Lines: 6
NNTP-Posting-Host: phakt.usc.edu






      

misc.fo

0

**¿Con qué clases se ha confundido más dicha consulta?**

Como se ve en la celda de arriba estas son las clases con las que mas se confunde puesto que son las clases de los mensajes con mayor similitud de coseno y que no son de la clase 
- talk.politics.guns
- rec.autos
- rec.sport.hockey
- misc.forsale
- alt.atheism
- alt.atheism
- rec.sport.baseball
- talk.politics.mideast

**¿A qué crees que se deben los malos resultados?**

A la utilizacion de una gran variedad de palabras que se puedan relacionar con varias categorias y al solapamiento de estas, por ejemplo entre política, armas y religión (atheism), en los que se tratan topicos con un léxico similar

## Apartado B

- Usamos un TF-IDF para ponderar el peso de los terminos de la bolsa de palabras
- Dicho TF-IDF lo transformamos con countVectorizer(binary = False) y aplicamos TfidfTransformer

In [16]:
vectorizerB = CountVectorizer(vocabulary=dictionary, stop_words='english',ngram_range=(1,1), binary=False)  
tfidfer = TfidfTransformer()

vectorDataB = vectorizerB.fit_transform(train_data.data)
vectorMod = tfidfer.fit_transform(vectorDataB)

In [18]:
#Calculamos la precision media como en el apartado anterior pero en este caso en la bolsa de palabras hemos modulado 
#la frecuencia segun lo popular que es el termino en train
for i in range (len(fetch_20newsgroups().target_names)):
    #Calculamos la distancia del coseno entre train_vector_data y get3message(target_names), es decir, la distancia
    #del coseno entre los mensajes de entrenamiento y los de consulta
    test = cosine_similarity(vectorMod, vectorizer.transform(get3message(fetch_20newsgroups().target_names[i])))
    print("Categoria: " + fetch_20newsgroups().target_names[i])
    print("Precision con nivel exhaustividad 10: "+str(calculoPrecision(10,fetch_20newsgroups().target_names[i],test)))
    print("Precision con nivel exhaustividad 3: "+str(calculoPrecision(3,fetch_20newsgroups().target_names[i],test)))
    print("---------------------------")

Categoria: alt.atheism
Precision con nivel exhaustividad 10: 80.0
Precision con nivel exhaustividad 3: 100.0
---------------------------
Categoria: comp.graphics
Precision con nivel exhaustividad 10: 70.0
Precision con nivel exhaustividad 3: 100.0
---------------------------
Categoria: comp.os.ms-windows.misc
Precision con nivel exhaustividad 10: 100.0
Precision con nivel exhaustividad 3: 100.0
---------------------------
Categoria: comp.sys.ibm.pc.hardware
Precision con nivel exhaustividad 10: 70.0
Precision con nivel exhaustividad 3: 100.0
---------------------------
Categoria: comp.sys.mac.hardware
Precision con nivel exhaustividad 10: 80.0
Precision con nivel exhaustividad 3: 100.0
---------------------------
Categoria: comp.windows.x
Precision con nivel exhaustividad 10: 50.0
Precision con nivel exhaustividad 3: 100.0
---------------------------
Categoria: misc.forsale
Precision con nivel exhaustividad 10: 70.0
Precision con nivel exhaustividad 3: 100.0
---------------------------

**¿Han cambiado los valores de precisión media para las clases del conjunto de datos? ¿Qué clases han mejorado? ¿Cuáles han empeorado?**

Han cambiado notablemente los valores de precisión media, en concreto han mejorado: 
- comp.os.ms-windows.misc
- comp.sys.ibm.pc.hardware
- comp.windows.x
- misc.forsale
- rec.sport.hockey
- sci.crypt
- soc.religion.christian
- talk.politics.guns (notablemente)
- talk.religion.misc
Han empeorado:
- talk.politics.mideast

---

La consulta en talk.politics.guns ha mejorado de un 30% a un 70%, esto es debido a que TF/IDF elimina palabras de uso comun no ligadas al contexto de la categoria en la que se encuentre, esto implica que sea mas facil diferenciar categorías con léxicos similares, como le ocurre a guns con otras categorias como por ejemplo "atheism"